In [2]:
import logging
from data_loader import load_data
from data_cleaning import clean_data
from data_transformation import transform_data
from feature_engineering import transform_and_split_data
import pandas as pd
import scipy.sparse
from pathlib import Path

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

OUTPUT_DIR = Path().resolve().parent.parent.joinpath("data", "processed")

def save_data(X_train, X_val, X_test, y_train, y_val, y_test, output_dir=OUTPUT_DIR):
    # Guarda las variables objetivo (y) como CSV
    y_train.to_csv(f"{output_dir}/y_train.csv", index=False)
    y_val.to_csv(f"{output_dir}/y_val.csv", index=False)
    y_test.to_csv(f"{output_dir}/y_test.csv", index=False)

    # Guarda los features (X) como Parquet o CSV
    if isinstance(X_train, scipy.sparse.spmatrix):
        scipy.sparse.save_npz(f"{output_dir}/X_train.npz", X_train)
        scipy.sparse.save_npz(f"{output_dir}/X_val.npz", X_val)
        scipy.sparse.save_npz(f"{output_dir}/X_test.npz", X_test)
    else:
        pd.DataFrame(X_train).to_parquet(f"{output_dir}/X_train.parquet")
        pd.DataFrame(X_val).to_parquet(f"{output_dir}/X_val.parquet")
        pd.DataFrame(X_test).to_parquet(f"{output_dir}/X_test.parquet")

    logging.info(f"✅ Datos guardados en {output_dir}")


# def run_data_preprocessing_pipeline()

logging.info("Iniciando pipeline de preprocesamiento...")

logging.info(f"📊 1. Carga de datos.")
data = load_data("clasificacion_siniestros.csv")
logging.info(f"✅ Datos cargados: {data.shape[0]} filas y {data.shape[1]} columnas.")

logging.info(f"🧹 2. Limpieza de datos.")
data = clean_data(data.copy())
logging.info(f"✅ Proceso de limpieza finalizado.")


2025-03-02 11:46:37,248 - INFO - Iniciando pipeline de preprocesamiento...
2025-03-02 11:46:37,250 - INFO - 📊 1. Carga de datos.
2025-03-02 11:46:37,251 - INFO - 	Cargando datos desde: D:\OneDrive - Universidad de La Salle\Maestría IA\S4\Desarrollo de soluciones\Proyecto\Repo desarrollo soluciones\data\raw\clasificacion_siniestros.csv
2025-03-02 11:46:38,308 - INFO - ✅ Datos cargados: 63164 filas y 66 columnas.
2025-03-02 11:46:38,308 - INFO - 🧹 2. Limpieza de datos.
2025-03-02 11:46:38,329 - INFO - 	Iniciando limpieza: (63164, 66) registros.
2025-03-02 11:46:38,781 - INFO - 	Columnas eliminadas por alto porcentaje de nulos (>50%): ['id_act_economica_igdacmlmasolicitudes', 'fecha_muerte_igatepmafurat', 'muerte_posterior_igatepmafurat', 'fecha_aviso_muerte_igatepmafurat']
2025-03-02 11:46:38,914 - INFO - 	Total registros eliminados por outliers: 2644.
2025-03-02 11:46:39,240 - INFO - 	Limpieza completada: (54638, 22) registros (cambio de (63164, 66) a (54638, 22)).
2025-03-02 11:46:39,2

In [ ]:
import pandas as pd
import numpy as np
import logging

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")


# def transform_data(df: pd.DataFrame) -> pd.DataFrame:

df=data.copy()

# 1. Extraer mes y día del siniestro de la columna 'fecha_siniestro_igdacmlmasolicitudes'
df['fecha_siniestro_month'] = df['fecha_siniestro_igdacmlmasolicitudes'].dt.month
df['fecha_siniestro_day'] = df['fecha_siniestro_igdacmlmasolicitudes'].dt.day

df.drop(columns=['fecha_siniestro_igdacmlmasolicitudes'], inplace=True)

logging.info(f"\tSe han extraído las variables temporales 'mes' y 'día' del siniestro.")

# 2. Remapear valores de la columna 'ind_realizando_trabajo_hab_at_igatepmafurat'
df['ind_realizando_trabajo_hab_at_igatepmafurat'] = df['ind_realizando_trabajo_hab_at_igatepmafurat'].map(
    {"si":"SI",
    "s":"SI",
    "sin informacion":"SIN INFORMACION",
    "no":"NO",
    "n":"NO",
    "1":"SI"})

logging.info(f"\tSe han remapeado los valores de la columna 'ind_realizando_trabajo_hab_at_igatepmafurat'.")

# 3. Ajuste de columnas de categoría
# Identificar columnas que empiezan con "ind" o "id"
cols_to_str = df.filter(regex='^(ind|id|emp|tipo|seg)').columns


# Convertir las columnas seleccionadas a tipo str
df[cols_to_str] = df[cols_to_str].astype(str)

# Variable a predecir como categórica
df['origen_igdactmlmacalificacionorigen'] = df['origen_igdactmlmacalificacionorigen'].astype(str)

logging.info(f"\tSe han ajustado las columnas de categoría.")

# 4. Capturar periodicidad en variables temporales

# Ciclo fijo de 12 meses (1-12)
df["fecha_siniestro_month_sin"] = np.sin(2 * np.pi * (df["fecha_siniestro_month"] - 1) / 12)
df["fecha_siniestro_month_cos"] = np.cos(2 * np.pi * (df["fecha_siniestro_month"] - 1) / 12)

# Ciclo variable de días según el mes (1-31)
dias_por_mes = np.array([
    31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31
])

# Mapear días máximos según el mes
dias_maximos = df["fecha_siniestro_month"].map(lambda x: dias_por_mes[x - 1])

# Calcular seno y coseno ajustado al mes
df["fecha_siniestro_day_sin"] = np.sin(2 * np.pi * (df["fecha_siniestro_day"] - 1) / dias_maximos)
df["fecha_siniestro_day_cos"] = np.cos(2 * np.pi * (df["fecha_siniestro_day"] - 1) / dias_maximos)

# Ciclo fijo de 24 horas (0-23)
df["hora_siniestro_sin"] = np.sin(2 * np.pi * df["hora_at_igatepmafurat"] / 24)
df["hora_siniestro_cos"] = np.cos(2 * np.pi * df["hora_at_igatepmafurat"] / 24)


# Ciclo fijo de 24 horas (0-23)
df["hora_previo_sin"] = np.sin(2 * np.pi * df["horas_previo_at_igatepmafurat"] / 24)
df["hora_previo_cos"] = np.cos(2 * np.pi * df["horas_previo_at_igatepmafurat"] / 24)


# Remover columnas originales
df.drop(columns=["fecha_siniestro_month", "fecha_siniestro_day", "hora_at_igatepmafurat","horas_previo_at_igatepmafurat"], inplace=True)

logging.info(f"\tSe han capturado las periodicidades en las variables temporales con seno y coseno.") 


# reemplazar valores fuera de s y n 
columnas_1 = ['dto_igdacmlmasolicitudes', 'pcl_igdacmlmasolicitudes']
columnas_2 = ['accidente_grave_igatepmafurat', 'riesgo_biologico_igatepmafurat']

# Aplicar reemplazos en las columnas correspondientes
df[columnas_1] = df[columnas_1].replace('', 'n')
df[columnas_2] = df[columnas_2].replace('0', 'n')


logging.info(f"\tSe han imputado los espacios en variables binarias.") 






2025-03-02 11:46:39,328 - INFO - 	Se han extraído las variables temporales 'mes' y 'día' del siniestro.
2025-03-02 11:46:39,336 - INFO - 	Se han remapeado los valores de la columna 'ind_realizando_trabajo_hab_at_igatepmafurat'.
2025-03-02 11:46:39,735 - INFO - 	Se han ajustado las columnas de categoría.
2025-03-02 11:46:40,296 - INFO - 	Se han capturado las periodicidades en las variables temporales con seno y coseno.
2025-03-02 11:46:40,298 - INFO - 	Se han imputado los espacios en variables binarias.


In [ ]:
df

In [ ]:

logging.info(f"🔄 3. Transformación de datos.")
data = transform_data(data.copy())
logging.info(f"✅ Proceso de transformación finalizado.")

logging.info(f"🔧 4. Ingeniería de características y partición de datos")
x_train, y_train, x_val, y_val, x_test, y_test = transform_and_split_data(data.copy())
logging.info(f"✅ Proceso de ingeniería de características y partición de datos finalizado.")

logging.info(f"📦 5. Guardando datos procesados.")
save_data(x_train, x_val, x_test, y_train, y_val, y_test)

